In [30]:
!pip install opendatasets
!pip install pandas
!pip install torchmetrics

In [117]:
import opendatasets as od 
import pandas 

od.download( 
    "https://www.kaggle.com/datasets/msambare/fer2013/data") 


Skipping, found downloaded files in "./fer2013" (use force=True to force download)


In [118]:
import os

# List the files in the downloaded folder
dataset_path = "fer2013"
os.listdir(dataset_path)

['train', 'test']

In [119]:
import torch
import torch.nn as nn
from torch.optim import Adam

# Checking if CUDA is available
flag_cuda = torch.cuda.is_available()

if not flag_cuda:
    print('Using CPU')
else:
    print('Using GPU')


Using GPU


In [194]:

from torchvision import transforms
import numpy as np
import random as random
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
# transform = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),  # Ensure single-channel grayscale
#     transforms.Resize((48, 48)),  # Resize to match FER2013 dimensions
#     transforms.ToTensor(),  # Convert PIL image to PyTorch tensor ✅
#     transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values
# ])

# valid_transform = transforms.Compose([
#     transforms.Grayscale(num_output_channels=1),  # Ensure single-channel grayscale
#     transforms.Resize((48, 48)),  # Resize to match FER2013 dimensions
#     transforms.ToTensor(),  # Convert PIL image to PyTorch tensor ✅
#     transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize pixel values
# ])

from torch.utils.data import DataLoader
from torchvision import datasets

from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
    
])

# Load full training set
full_dataset = datasets.ImageFolder(root="fer2013/train", transform=transform)

print(full_dataset)

# Split 80/20
train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size
train_dataset, valid_dataset = random_split(full_dataset, [train_size, valid_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

# Test set
test_dataset = datasets.ImageFolder(root="fer2013/test", transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Dataset ImageFolder
    Number of datapoints: 28709
    Root location: fer2013/train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=[0.5], std=[0.5])
           )


In [184]:
# Dataloaders
# batch_size = 32
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [185]:
# # Get dataset sizes
# train_N = len(train_dataset)
# valid_N = len(valid_dataset)

In [186]:

# Hyperparameters
n_classes = 7  # FER2013 has 7 emotions
IMG_CHS = 1  # Grayscale images
kernel_size = 3
flattened_img_size = 75 * 3 * 3

# Define the CNN Model
model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 48 x 48
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 24 x 24
    
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 24 x 24
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 12 x 12
    
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 12 x 12
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.MaxPool2d(2, stride=2),  # 75 x 6 x 6
    
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(75 * 6 * 6, 512),
    nn.Dropout(0.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)  # Output 7 classes
)


In [187]:
device = torch.device("cuda" if flag_cuda else "cpu")
model = model.to(device)


In [188]:
# Define Loss and Optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [189]:
# Function to calculate batch accuracy
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [190]:
# Validation function
def validate():
    loss = 0
    accuracy = 0
    model.eval()
    valid_N = len(valid_loader.dataset)
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    
    print(f'Valid - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')

In [191]:
# Training function
def train():
    loss = 0
    accuracy = 0
    model.train()
    train_N = len(train_loader.dataset)
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    
    print(f'Train - Loss: {loss:.4f} Accuracy: {accuracy:.4f}')


In [192]:
import torchmetrics
def compute_f1():
    model.eval()
    f1_metric = torchmetrics.F1Score(task="multiclass", num_classes=7).to("cuda")
    f1_metric.reset()
    
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)  # Keep everything on GPU
            output = model(x)
            predictions = torch.argmax(output, dim=1)

            f1_metric.update(predictions, y)  # Directly update the metric
    print(f"F1_score: {f1_metric.compute().item():.4f}")


In [193]:
# Training loop
epochs = 20

for epoch in range(epochs):
    print(f'Epoch: {epoch+1}/{epochs}')
    train()
    validate()
    compute_f1()

Epoch: 1/20
Train - Loss: 598.9866 Accuracy: 0.3469
Valid - Loss: 136.6679 Accuracy: 0.4441
F1_score: 0.4441
Epoch: 2/20
Train - Loss: 514.4733 Accuracy: 0.4467
Valid - Loss: 122.6078 Accuracy: 0.4906
F1_score: 0.4906
Epoch: 3/20
Train - Loss: 478.4685 Accuracy: 0.4884
Valid - Loss: 115.5131 Accuracy: 0.5024
F1_score: 0.5024
Epoch: 4/20
Train - Loss: 456.7639 Accuracy: 0.5140
Valid - Loss: 112.3119 Accuracy: 0.5204
F1_score: 0.5204
Epoch: 5/20
Train - Loss: 436.8511 Accuracy: 0.5367
Valid - Loss: 109.6250 Accuracy: 0.5366
F1_score: 0.5366
Epoch: 6/20
Train - Loss: 421.3740 Accuracy: 0.5575
Valid - Loss: 106.9106 Accuracy: 0.5524
F1_score: 0.5524
Epoch: 7/20
Train - Loss: 410.6085 Accuracy: 0.5675
Valid - Loss: 106.9850 Accuracy: 0.5458
F1_score: 0.5458
Epoch: 8/20
Train - Loss: 396.8216 Accuracy: 0.5780
Valid - Loss: 104.6621 Accuracy: 0.5543
F1_score: 0.5543
Epoch: 9/20
Train - Loss: 384.3249 Accuracy: 0.5904
Valid - Loss: 102.8100 Accuracy: 0.5594
F1_score: 0.5594
Epoch: 10/20
Train 